

# <center>Projeto DataView





### Foi projetado para ter 4 tipos de consultas

* A primeira é para receber dados dos professores que estão resgistrados no sistema

* A segunda é para receber dados das turmas que estão registradas no sistema

* A terceira é para receber dados das submissões que estão registradas no sistema

* A quarta é para receber dados das questões que estão registradas no sistema

As consultas vão ser tornando acumulativas, com a segunda dependendo de dados da primeira, e a terceira dependendo de dados da primeira e segunda para o preenchimento da url de consulta. Todas as consultas dependem da key de acesso

Na consulta da class possui as depedências de: 
1. id_teacher, que é o id de um professor
2. year, que é o ano da turma 
3. semester, que é o semestre (1 ou 2) da turma

E na consulta de submission necessita do:
1. id_class, que é o id de uma turma.

Já a consulta de question e do teacher necessita apenas da key

As consultas serão feitas via protocolo HTTP usando o método Get e irão receber um JSON dos dados do LoP


### Estrutura das funções e suas dependências

Funções de coleta e transformações de dados 

- lop_class(), lop_question()
--- lop_submission()
------ question_data()
--------- performance_list()
------------ graph_more_less_70_list_class()
------------ graph_distribution_notes_list()

Funções de treinamento de modelos de Machine Learning

### Descrição dos Dados
#### <center> Dataframe Submissões
| **Semestre** | **Número de atributos (colunas)** | **Número de Registros (linhas)** |
| --- | --- | --- |
| 2020.5 | xxxx | yyy |
| 2020.6 | yyyy | xxx |

#### <center>Dataframe Resultados em LOP
| **Semestre** | **Número de atributos (colunas)** | **Número de Registros (linhas)** |
| --- | --- | --- |
| 2020.5 | xxxx | yyyyy |


---

#### Descrição do datafrane dos professores - Consulta embutida em lop_class()

<b>1. id_teacher</b> ID do professor no LoP

<b>2. email</b> Email do professor

<b>3. name_teacher</b> Nome do professor

--- 

#### Descrição do dataframe das turmas - lop_class()

<b>1. id_teacher</b> ID do professor no LoP

<b>2. id_class</b> ID da turma

<b>3. name_class</b> Nome da turma

<b>4. code</b> Código de acesso a turma no LoP

<b>5. year</b> Ano em que a turma foi cadastrada

<b>6. semester</b> Semestre em que a turma foi cadastrada

---
#### Descrição dos dataframe de Submissão - lop_submission()


<b>1. environment</b> Tipo de máquina

<b>2. hitPercentage</b> Porcentagem de acerto na questão

<b>3. language</b> Linguagem de programação usada

<b>4. char_change_number</b> Número de caracteres alterados

<b>5. timeConsuming</b> Tempo que passou na questão

<b>6. createdAt</b> Data e hora da submissão

<b>7. user</b> Nome e matrícula do aluno

<b>8. question</b> Nome da questão

<b>9. list</b> Nome da lista

<b>10. test</b> Nome da prova

<b>11. id_class</b> ID da turma em que o aluno está associado nessa submissão

---

#### Descrição do dataframe das questões do LoP - lop_question()


<b>1. id_questao</b> ID da questão

<b>2. question</b> Nome da questão

<b>3. difficulty </b> Nível de dificuldade da questão (1-5)

<b>4. tags </b> Esse campo vem com um json que contem os assuntos em que essa questão está associada

<b>5. lists </b> Esse campo vem com um json que contem as listas que a questão está associada 

<b>6. tests</b> Esse campo vem com um json que contem as provas que a questão está associada 

---

#### Descrição do dataframe de dados das questões - question_data()


<b>1. id_list</b> ID da lista

<b>2. list</b> Nome da lista

<b>3. question </b> Nome da questão

<b>4. difficulty </b> Nível de dificuldade da questão (1-5)

<b>5. tags </b> Em cada coluna com nome tag, terá um conteúdo associado a ela

---


#### Descrição do dataframe de performance por lista - performance_list()


<b>1. user</b> Nome e matrícula do aluno

<b>2. list</b> Nome da lista

<b>3. totalHitPercentage </b> Soma de todos acertos de questões únicas

<b>4. totalQuestionsList </b> Número de questões por lista

<b>5. mediaList</b> Esse campo vem com um json que contem as provas que a questão está associada 

---

## 1. Importação das bibliotecas
<a id='importacao'></a>

In [1]:
import pandas as pd
import numpy as np
import json
import urllib3
urllib3.disable_warnings()
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re

## 2. Funções
<a id='funcoes'></a>

### 2.1 Funções de consulta e transformação

Função de leitura de arquivos txt

In [2]:
def read_txt(path):
  with open(path,'r') as file:
    return file.read()

Função de consulta no LoP

In [3]:
def lop_consult(url):
  http = urllib3.PoolManager()
  req = http.request('GET',url)
  if req.status == 200:
    return pd.read_json(req.data, orient = 'RECORDS', encoding = 'utf-8').copy()
  else: 
    return False

Função para obter todas as classes do LoP a partir de uma lista de professores

In [4]:
def lop_class(df_teacher = pd.DataFrame()):
  #Essa função se passar por cópia o df_teacher se torna desnecessário fazer a requisição de todas os professores, e evitando fazer varias consultas
  #Dataframe onde será armazenado todas as turmas
  df_class = pd.DataFrame()
  if df_teacher.empty == True:
    #Formação da url de consulta para saber os professores cadastrados no LoP
    url_teacher = endpoint_teacher + key
    #Realizando a consulta no LoP
    df_teacher = lop_consult(url_teacher)#id, email, name
  #O laço faz uma varredura das turmas de cada professor e por fim armazena em um dataframe de turmas
  for id_teacher, name_teacher in zip(df_teacher['id'],df_teacher['name']):
    #O LoP.v2 foi desenvolvido em 2019, então temos que capturar as turmas entre 2019 e o ano atual
    current_year = datetime.now().year
    for year in range(2019, current_year + 1):
      #Por fim precisamos do semestre que deve ser 1 ou 2
      for semester in range(1,3): 
        try:
          #Url de consulta de classes
          url_class = endpoint_class + id_teacher + '?year=' + str(year) + '&semester=' + str(semester) + '&key=' + key 
          #Dataframe provisório necessário para capturar o tamanho e saber quantas turmas esse professor tem que estar associado
          df_prov = lop_consult(url_class)
          #Inserindo o id e o nome do professor no dataframe de turmas
          df_prov['id_teacher'] = id_teacher
          df_prov['name_teacher'] = name_teacher
          #Adicionando turmas conforme novas consultas
          df_class = df_class.append(df_prov, ignore_index=True).copy()#id, name, code, year, semester, id_teacher
        except:
          pass
  #Renomeando colunas com nomes semelhantes em todos os DF
  df_class.rename(columns={'id':'id_class','name':'name_class'}, inplace = True)
  return df_class

Função para obter todas as submissões a partir de uma lista de turmas

In [5]:
def lop_submission(df_class = pd.DataFrame()):
  #Essa função se passar por cópia o df_class se torna desnecessário fazer a requisição de todas as turmas usando a função lop_class()
  #Dataframe onde serão armazenados todas as submissões 
  df_submission = pd.DataFrame()  
  if df_class.empty == True:
    df_class = lop_class()  
  #Obtem todas as submissões da lista de turmas passadas
  for id_class in df_class['id_class']:
    #Url da consulta as submissões
    url_submission = endpoint_submission + id_class + '/submission?key=' + key
    #Dataframe provisório que vai armazenar o resultado da consulta
    df_prov = lop_consult(url_submission)
    #Inserindo o id_class no dataframe de consultas
    df_prov['id_class'] = id_class
    #Adicionando submissões conforme consultas
    df_submission = df_submission.append(df_prov, ignore_index=True).copy()
  #Campo user trás matricula e o nome do usuário, vamos separar em 2 colunas distintas
  df_submission[['user','registration']] = df_submission['user'].str.split('-',expand=True)
  return df_submission

Função para obter dados das questões cadastradas no LoP

In [6]:
def lop_question():
  url_question = endpoint_question + key
  df_lop_question = lop_consult(url_question)
  df_lop_question.rename(columns = {'id':'id_question','title':'question'}, inplace = True)
  return df_lop_question

Função que transforma os dados das questões extraídas do LoP, assim podemos saber a dificuldade de uma questão, que pode estar associada a uma ou mais listas

In [7]:
def question_data(df_question = pd.DataFrame()):
  #Essa função se passar por cópia o df_question se torna desnecessário fazer a requisição de todas as turmas usando a função lop_question()
  #Dataframes onde serão armazenados todas as questões transformadas
  df_list_data = pd.DataFrame() 
  df_test_data = pd.DataFrame() 
  df_tag_data = pd.DataFrame()  
  df_question_data = pd.DataFrame()  
  #i=0
  if df_question.empty == True:
    df_question = lop_question()
  for question,lists,tests,tags,difficulty in zip(df_question['question'],df_question['lists'],df_question['tests'],df_question['tags'],df_question['difficulty']):
    #Cada registro do json pode ficar com mais de uma lista associada a uma questão, por isso a necessidade de extrair
    df_prov_lists  = pd.DataFrame(json.loads(str(lists).replace("'",'"')))
    #Cada registro do json pode ficar com mais de uma lista associada a uma questão, por isso a necessidade de extrair
    df_prov_tests  = pd.DataFrame(json.loads(str(tests).replace("'",'"')))
    #df_prov_lists.rename(columns = {'id':'id_list','title':'list'}, inplace = True)
    #Inserindo a questão associada
    df_prov_lists['question'] = question
    df_prov_lists['difficulty'] = difficulty
    df_list_data = df_list_data.append(df_prov_lists, ignore_index = True).copy()
    #print(df_list_data['question'])
    #print("numero de questoes unicas no df ",len(df_list_data['question'].unique()))
    df_prov_tests['question'] = question
    df_prov_tests['difficulty'] = difficulty
    df_test_data = df_test_data.append(df_prov_tests, ignore_index = True).copy()
    #Extraindo as tags, que vão dizer o conteudo associado
    df_prov_tags = pd.DataFrame(json.loads(str(tags).replace("'",'"'))).T
    #Inserindo a questão associada
    df_prov_tags['question'] = question    
    df_tag_data = df_tag_data.append(df_prov_tags, ignore_index = True).copy()
    #print(i,' ------------------  ',question, ' ',difficulty)
    #i+=1
    #if i == 100:
    #  break
  
  df_list_data.rename(columns = {'id':'id_list','title':'list'}, inplace = True)
  df_test_data.rename(columns = {'id':'id_test','title':'test'}, inplace = True)
  #print(len(df_list_data['question'].unique()))
  #df_list_data['difficulty'] = df_list_data['difficulty'].astype('int')
  columns_tags = ['question']
  for i in range(len(df_tag_data.columns)-1): columns_tags.append('tag'+str(i+1))
  df_tag_data.columns = columns_tags
  df_list_data = pd.merge(df_list_data, df_tag_data, on = 'question', how = 'outer')#.fillna(0)
  df_test_data = pd.merge(df_test_data, df_tag_data, on = 'question', how = 'outer')#.fillna(0)
  return  pd.concat([df_list_data, df_test_data],axis=0).copy()

Função que consula no lop dados sobre as listas

In [8]:
def lop_lists(turma):#Turma como dataframe
  url_lists = endpoint_lists + data.iloc[0,0] + '/list/all?key=' + key
  df_lop_lists = lop_consult(url_lists)
  df_lop_lists.rename(columns = {'id':'id_list','title':'list'}, inplace = True)
  return df_lop_lists

### 2.2. Funções referentes a lista

Função para avaliar a perfomance por lista

In [9]:
def performance_list(df_submission):  
  #Essa função necessita receber as submissões de uma turma ---------- depois podemos incrementar essa função ao ter uma turma associada e agrupar por elas
  #Me traz as porcentagens maximas de acerto por questão
  df_performance_list = df_submission.groupby(['user','list','question'])['hitPercentage'].max().reset_index()
  #Soma as porcentagens de uma lista unica
  df_performance_list = df_performance_list.groupby(['user','list'])['hitPercentage'].sum().reset_index()
  #Renomeando para melhorar entendimento
  df_performance_list.rename(columns={'hitPercentage':'totalHitPercentage'}, inplace = True)
  #Usando a função de dados de questão para ter informações sobre que questões estão associadas a lista
  df_question_data = question_data()
  #Agrupa por lista e questão para ter apenas uma questão de cada, a operação não importa muito, apenas existe para completar o agrupamento --------- pode ser uma boa trocar por drop_duplicates
  df_question_data = df_question_data.groupby(['list','question'])['id_list'].count().reset_index()
  #Conta quantas questões tem por lista
  df_question_data = df_question_data.groupby(['list'])['question'].count().reset_index()
  #Renomeando lista
  df_question_data.rename(columns={'question':'totalQuestionsList'}, inplace = True)
  #Merge
  df_performance_list = df_performance_list.merge(df_question_data, on = 'list')
  #Média de acerto por lista
  df_performance_list['mediaList'] = df_performance_list['totalHitPercentage'] / df_performance_list['totalQuestionsList']
  return df_performance_list

Função do gráfico de barras que, por lista, faz a contagem de quem fez mais e menos de 70% da lista. Necessita de passar a performance do aluno por lista

In [10]:
def graph_more_less_70_list_class(df_performance_list):
  #Aqui eu conto, por lista, quantos tiraram mais de 70% de acerto
  df_more70 = df_performance_list[df_performance_list['mediaList'] >= 70.0].groupby('list')['mediaList'].count().reset_index(name='more70')
  #E nesse conta quantos tiveram menos de 70%
  df_less70 = df_performance_list[df_performance_list['mediaList'] < 70.0].groupby('list')['mediaList'].count().reset_index(name='less70')
  #Aqui junta ambos os dataframes, usando o outer, que, se por acaso não tiver uma ocorrência de uma lista em um dos dataframes, mesmo assim eles vão ser adicionados, é união dos conjuntos
  df_less_more_70 = pd.merge(df_more70, df_less70, on = 'list', how = 'outer')
  #E, se caso acontecer a situação de um não existir, substituimos o nan por 0
  df_less_more_70.replace(np.NaN, 0, inplace = True)
  return df_less_more_70.to_json('df_less_more_70.json',force_ascii=False, orient='records') #############3 por hora essa função tamebm esta realizando downlaod

Função do grafico de distribuição de notas por lista. Necessita da performance do  

In [11]:
def graph_distribution_notes_list(df_performance_list):
  #Agrupa por lista, e conta as ocorrências
  return df_performance_list.groupby('list')['mediaList'].apply(lambda group: group.values.tolist()).to_json('df_distribuition_notes_list.json',force_ascii=False,orient='index')

In [12]:
def select_questions(df_question_data,df_lop_lists):
  #Selecionando onde List não é NaN
  df_prov1 = df_question_data.dropna(subset=['list'])#.fillna(0)
  #Selecionando onde Test não é NaN
  #df_prov2 = df_question_data.dropna(subset=['test'])#.fillna(0)
  #Concatenação para permitir um loc que busque em todas as instancias
  pat1 = '|'.join(['({})'.format(re.escape(c)) for c in list(df_lop_lists['id_list'])])
  #pat2 = '|'.join(['({})'.format(re.escape(c)) for c in provas_escolhidas])
  #Busca
  df_prov1 = df_prov1.loc[df_prov1['id_list'].str.contains(pat1)].copy()
  #df_prov2 = df_prov2.loc[df_prov2['test'].str.contains(pat2)].copy()
  #return pd.concat([df_prov1, df_prov2],axis=0).copy()
  return df_prov1

### 2.3. Funções referentes a dificuldade

Função que faz o cálculo da performance dos alunos por dificuldade e retorna um dataframe. É necessário passar por parâmetro uma turma e uma variável bool com valor True(para Prova) ou False(para Lista)

In [13]:
def df_difficulty_performance (turma, prova = False):

  df_submissions = lop_submission(pd.DataFrame(turma, columns = ['id_class'])) # carregando dados de submissões da turma
  df_questions = select_questions(question_data(),listas_escolhidas,provas_escolhidas) # carregando dados de questões selecionadas
  
  if (prova): # filtro de provas 
    df_questions = df_questions.dropna(subset=['test']).fillna(0)
  else:
    df_questions = df_questions.dropna(subset=['list']).fillna(0)

  df_geral = pd.merge(df_submissions,df_questions, on = "question", how = "inner") # juntando os dataframes de submissões e questões
  df_difficulty_performance = df_geral.groupby(['user','registration','question','difficulty'])['hitPercentage'].max().reset_index() # pegando o máximo de HitPercentage de cada aluno
  df_difficulty_performance = df_difficulty_performance.groupby(['user','registration','difficulty'])['hitPercentage'].sum().reset_index() # fazendo a soma dos máximos
  df_difficulty_performance.rename(columns={'hitPercentage':'totalHitPercentage'},inplace = True) # renomeando coluna 'hitPercentage'

  df_questions = df_questions.groupby(['question','difficulty']).count().reset_index() # agrupando por questão e dificuldade
  df_questions = df_questions.groupby(['difficulty'])['question'].count().reset_index() # contando quantas questões por dificuldade existem
  df_questions.rename(columns={'question':'totalQuestions'},inplace=True) # renomeando coluna 'question'


  df_difficulty_performance = addData(df_difficulty_performance, df_difficulty_performance['difficulty'].unique(), df_difficulty_performance['user'].unique()  ) # chamando função para adicionar dados faltantes
  df_difficulty_performance = df_difficulty_performance.merge(df_questions, on = 'difficulty', how = 'outer') # juntando o dataframe principal com o de contagem de questões
  df_difficulty_performance.replace(np.NaN, 0 , inplace = True)
  df_difficulty_performance['averageDifficulty'] = df_difficulty_performance['totalHitPercentage'] / df_difficulty_performance['totalQuestions'] # adicionando coluna da média por dificuldade
  df_difficulty_performance = df_difficulty_performance.sort_values(['user','difficulty']) # ordernando o dataframe por nome e dificuldade

  return df_difficulty_performance

Função que retorna o json de estudantes aprovados, reprovados e faltosos de todas as listas passadas pelo professor. É necessário passar por parâmetro uma turma, a nota esperada para aprovação e uma variável bool com valor True(para Prova) ou False(para Lista)

In [14]:
def graph_aprov (turma, nota_esperada, prova = False):

  df_class = lop_class() # carregando dados de turmas

  if (prova): # filtro de provas
    performance_df = df_difficulty_performance(turma, True)
  else:
    performance_df = df_difficulty_performance(turma, False)

  df_aprov = performance_df[performance_df['averageDifficulty']>=nota_esperada].groupby('difficulty')['averageDifficulty'].count().reset_index(name  = 'approved') # contando quantos estudantes estão com a média acima do parâmetro de 'nota_esperada'
  df_disaprov= performance_df[performance_df['averageDifficulty']<nota_esperada].groupby('difficulty')['averageDifficulty'].count().reset_index(name = 'disapproved') # contando quantos estudantes estão com a média abaixo do parâmetro de 'nota_esperada'
  df_aprov_disaprov = pd.merge(df_aprov, df_disaprov, on='difficulty', how='outer') # juntando os dataframes de aprovados e reprovados
  df_aprov_disaprov.replace(np.NaN,0,inplace=True) # substituindo NaN por zeros
  df_aprov_disaprov = df_aprov_disaprov[['difficulty','approved','disapproved']] # reorganizando colunas do dataframe

  df_turma= df_class.query("id_class == {}".format(turma))[["id_class","studentsCount"]].drop_duplicates().reset_index().drop('index',axis=1) # buscando o numero de alunos matriculados na turma
  num_alunos = int(df_turma["studentsCount"][0]) # armazenando em uma variável o valor do numero de alunos matriculados na turma
  df_aprov_disaprov['missing'] = num_alunos - (df_aprov_disaprov['approved'] + df_aprov_disaprov['disapproved']) # cálculo de faltosos

  df_aprov_disaprov = df_aprov_disaprov.astype(int) # transformando todo o dataframe em int
  df_aprov_disaprov = df_aprov_disaprov.sort_values(by='difficulty') # ordernando por dificuldade
  df_aprov_disaprov= df_aprov_disaprov.replace({'difficulty': {1:"Muito Fácil",2:"Fácil", 3:"Médio",4:"Difícil",5:"Muito Difícil"} }) # renomeando as dificuldades para melhor entendimento
  return df_aprov_disaprov.to_json('df_aprov_disaprov.json', force_ascii=False, orient='records')

Função que retorna o json com as médias de cada aluno por dificuldade e também a média da turma naquela dificuldade especifica. É necessário a passagem por parâmetro de uma turma e uma variável bool com valor True(para Prova) ou False(para Lista)

In [15]:
def averagebyDifficulty (turma, prova=False):
    
    if (prova): # filtro de provas
        performance_df = df_difficulty_performance(turma, True)

    else:
        performance_df = df_difficulty_performance(turma, False)

    performance_df.drop(['totalHitPercentage','totalQuestions'],axis=1,inplace=True) # como essas colunas não serão usadas, descartei elas

    df_student_averagebyDifficulty = performance_df
    df_mean = df_student_averagebyDifficulty.groupby(['difficulty'])['averageDifficulty'].mean().reset_index() # agrupando por dificuldade e fazendo a média da coluna 'averageDifficulty'
    df_mean.rename(columns={'averageDifficulty':'classAverage'},inplace=True) # renomeando a coluna para melhor entendimento
    df_student_averagebyDifficulty = df_student_averagebyDifficulty.merge(df_mean, how='outer', on='difficulty') # juntando o df principal com o df da média da turma
    df_student_averagebyDifficulty = df_student_averagebyDifficulty.sort_values(['user','difficulty']) # ordernando por nome e dificuldade
    df_student_averagebyDifficulty.replace({'difficulty': {1:"Muito Fácil",2:"Fácil", 3:"Médio",4:"Difícil",5:"Muito Difícil"} }, inplace=True) #renomeando as dificuldades para melhor entendimento

    return df_student_averagebyDifficulty.to_json('df_student_averagebyDifficulty.json', force_ascii=False, orient='records')

Função que retorna o json com as média geral do aluno, referente a soma das médias que o aluno obteve em cada dificuldade sobre o número de dificuldades existentes. É necessário a passagem por parâmetro de uma turma e uma variável bool com valor True(para Prova) ou False(para Lista)

In [16]:
def averageAllDifficulty (turma, prova = False ):
    
    if (prova): # filtro de provas
        performance_df = df_difficulty_performance(turma, True)
    else:
        performance_df = df_difficulty_performance(turma, False)

    performance_df.drop(['totalHitPercentage','totalQuestions'],axis=1,inplace=True) # como essas colunas não serão usadas, descartei elas

    df_allmean = performance_df.groupby(['user','registration','difficulty','averageDifficulty']).count().reset_index() # agrupando por nome, matricula, dificuldade e média
    df_allmean = df_allmean.groupby(['user','registration'])['averageDifficulty'].mean().reset_index() # agrupando por nome e matricula e fazendo a média da turma
    df_allmean.rename(columns={'averageDifficulty':'averageAllDifficulty'},inplace=True) # renomeando a coluna para melhor entendimento
    return df_allmean.to_json('df_averageAllDifficulty.json', force_ascii=False, orient='records')

### FUNÇÕES TEMPORÁRIAS

Função para adicionar dados faltoso ao dataframe de performance por dificuldade

In [17]:
def addData (df_difficulty_performance, difficulties, students):
  new_obs = pd.DataFrame()

  for i in df_difficulty_performance.columns.tolist():
    new_obs[i]= [np.nan] # preenchendo todas as colunas com NaN
  
  for student in students:
    new_obs["user"] = [student] # modifica o user da primeira linha para o estudante em questao
    df_student = df_difficulty_performance[df_difficulty_performance["user"] == student].reset_index().drop('index', axis=1) # filtra apenas pro aluno em questao
    diff_questions = list(set(df_student["difficulty"].unique()) ^ set(difficulties)) # cria lista com dificuldades nos quais não houveram submissao do aluno
    
    if len(diff_questions)>0: # caso haja alguma dificuldade que o aluno não submeteu
      try:
        registration = df_student["registration"].iloc[0] # adiciona matricula
      except:
        break
      for difficulty in diff_questions:
        new_obs["difficulty"] = [difficulty] # modifica dificuldade
        new_obs["registration"] = [registration] # modifica a matricula
        df_difficulty_performance = df_difficulty_performance.append(new_obs, ignore_index = True) # adiciona uma linha ao df original com a informação de user, registration e dificuldade
  return df_difficulty_performance

## 3. Leitura da Base
<a id='leiturabase'></a>
### 3.1. Leitura de endpoints e key

In [18]:
key = read_txt('key.txt')
endpoint_teacher = read_txt('endpoint_teacher.txt')
endpoint_class = read_txt('endpoint_class.txt')
endpoint_submission = read_txt('endpoint_submission.txt')
endpoint_question = read_txt('endpoint_question.txt')     
endpoint_lists = read_txt('endpoint_lists.txt')

## 4. Sandbox 

Nessa seção é feito uma exploração dos dados para criação de funções

In [19]:
listas_escolhidas = ['Laboratório 01 - Expressões Aritméticas',

'Aula - Introdução Expressões Aritiméticas',

'Aula - Funções Aritméticas',

'Treinamento - Expressões Aritméticas',

'(Lop) Lista de Laboratório 2 - Estrutura de decisão',

'(Lop) Estruturas de decisão - problemas sobre divisibilidade',

'(Lop) Estruturas de decisão - Múltiplas decisões',

'(Lop) Estruturas de decisão - Operadores Lógicos',

'Repetição condicional - Lista Resolvida (LOP)',

'Repetição condicional - Lista Prática (LOP)',

'Repetição condicional - Lista de Exercícios (LOP)',

'Repetição contada - Lista Resolvida (LOP)',

'Repetição contada - Lista Prática (LOP)',

'Repetição contada - Lista de Exercícios (LOP)',

'Vetores - Lista Resolvida (LOP)',

'Vetores - Lista de Exercícios (LOP)',

'Vetores - Lista Prática (LOP)']

provas_escolhidas = ['Prova LoP - Turma 05 - 2020.6']

In [104]:
turma = ['8394d1cb-86b5-4b90-bc62-4b9b192a81f6']

In [ ]:
lop_class().to_csv('df_class.csv')
df_class = pd.read_csv('df_class.csv')

In [ ]:
data = pd.DataFrame(['8394d1cb-86b5-4b90-bc62-4b9b192a81f6'], columns = ['id_class'])

Preparando os dados que serão utilizados

O objetivo da sprint dessa vez é estudar quanto cada questão demora para ser resolvida, a ideia é pegar o tempo máximo de tempo que um único usuário levou para resolver a questão e também o número máximo de dias diferentes que um usuário submeteu essa questão. 

Colunas do Dataframe:

Questão | Dificuldade | TempoMaximoGasto | MaximoDiasDiferentesSubmissão

In [21]:
df_submission = lop_submission(pd.DataFrame(turma, columns = ['id_class']))

In [22]:
#df_questions = select_questions(question_data(),listas_escolhidas,provas_escolhidas)
df_questions = question_data()

In [23]:
df_questionsCount = df_questions.groupby(['question','list']).count().reset_index()
df_questionsCount = df_questionsCount.groupby(['list'])['question'].count().reset_index()

In [24]:
df_submission = df_submission.merge(df_questions[:][['question','difficulty']], on = 'question', how = 'inner')
#df_submission[:]['tag1'].replace(np.NaN, 'ASSUNTO_EM_BRANCO', inplace= True)

In [25]:
colunasDif = ['timeConsuming','createdAt','question','difficulty','user','registration']

In [26]:
df_submission = df_submission[df_submission.timeConsuming>0][colunasDif].sort_values(by=['difficulty'])

Desenvolvendo a coluna de máximo de dias diferentes para cada questão

In [27]:
from datetime import datetime

In [28]:
df_submission['createdAt'] = pd.to_datetime(df_submission['createdAt'])

In [29]:
df_submission['dateSubmission'] = df_submission['createdAt'].dt.date

In [30]:
df_teste = df_submission[df_submission.timeConsuming>0].drop(['createdAt','timeConsuming'],axis =1).drop_duplicates()

In [31]:
df_teste = df_teste.groupby(['user','registration','question','difficulty']).count().reset_index()

In [32]:
df_maxDifferentDays = df_teste[:][['question','difficulty','dateSubmission']].groupby(['question','difficulty']).max().sort_values(by=['difficulty','dateSubmission']).rename(columns = {'dateSubmission':'MaxDifferentDays'}).reset_index()

Desenvolvendo a coluna de máximo de tempo gasto para cada questão

In [33]:
df_teste2 =df_submission[df_submission.timeConsuming>0].drop(['createdAt','dateSubmission'], axis =1)
df_teste2 ['timeConsuming'] = df_teste2['timeConsuming'].divide(1000).astype(int)

In [34]:
df_teste2 = df_teste2.groupby(['user','registration','question','difficulty']).sum().reset_index()

In [35]:
df_maxTimeConsuming = df_teste2.groupby(['question','difficulty']).max().drop(['user','registration'],axis=1).sort_values(by=['difficulty','timeConsuming']).rename(columns = {'timeConsuming':'maxTimeConsumingInSeconds'}).reset_index()

In [36]:
 #merge dos dois dfs
 df_maxConsuming = pd.merge(df_maxTimeConsuming, df_maxDifferentDays[['question','MaxDifferentDays']], on = 'question')
 

In [37]:
df_maxConsuming#.to_json('df_MaxConsuming.json', force_ascii=False, orient= 'records')

,question,difficulty,maxTimeConsumingInSeconds,MaxDifferentDays
0,Contando de 2 em 2,1.0,699,2
1,Duas ou mais condições com &&,1.0,1090,1
2,Categorias de natação com 4 casos,1.0,1566,1
3,Preenchimento de vetor II,1.0,1629,1
4,Calculando expressões - parte 3,1.0,1874,4
...,...,...,...,...
121,Qual o dia da semana?,4.0,84591,8
122,Raízes de uma equação de segundo grau,4.0,97242,4
123,Divisível por 3 e por 4 - versão mais complexa,4.0,158848,4
124,Soma de fatoriais,5.0,22966,3


In [38]:
graph_aprov(turma, 70, False)

In [39]:
df_questions = select_questions(question_data(),listas_escolhidas,provas_escolhidas)

In [40]:
df_questions = df_questions.groupby(['question','difficulty']).count().reset_index() # agrupando por questão e dificuldade
df_questions = df_questions.groupby(['difficulty'])['question'].count().reset_index() # contando quantas questões por dificuldade existem
df_questions.rename(columns={'question':'totalQuestions'},inplace=True) # renomeando coluna 'question'

In [41]:
df_questions

,difficulty,totalQuestions
0,1.0,36
1,2.0,49
2,3.0,33
3,4.0,6
4,5.0,2


In [43]:
a= lop_class()

In [73]:
max(a['id_class'], key=len)

'bee7d92a-141d-4657-aeed-b3a2fc60b026'

In [72]:
len(max(a['id_class'], key=len))

36

In [ ]:
create TABLE lopClass(
    id_teacher varchar(36) NOT NULL, -- o maximo foi 36
    name_teacher TEXT NOT NULL, -- o maximo foi 37
    name_class varchar(56) NOT NULL,  -- o maximo foi 56
    code varchar(10) NOT NULL, -- o maximo foi 10
    year NUMERIC(5,1) NOT NULL, -- o ano sempre vem no formato '20xx.0'
    semester NUMERIC(2,1) NOT NULL, -- o semestre é sempre especificado como '1.0','2.0' etc
    studentsCount NUMERIC(4,1) NOT NULL, -- o numero de estudantes, coloquei de tamanho 4 pois as turmas não passam de 130 alunos
    teachersCount NUMERIC(2,1) NOT NULL 
);

### Dissecando funções
A ideia aqui é estudar as otimizações que outros membros fizeram para funções feitas por mim, com o intuito de entender melhor cada otimização e me atualizar de métodos mais eficientes.

| Funções para estudar | Estudadas |
| :--------------------------------------------------- | : --------------------: |
| | |
| | |
| | |
| performance_difficulty_list_test | 1 |
| graph_more_less_difficulty_list_test_class | 0 |
| graph_performance_student_difficulty_list_test | 0 |
| media_graph_performance_student_difficulty_list_test | 0 |

In [59]:
def select_questions(df_question_data,df_lop_lists):
  #Selecionando onde List não é NaN
  df_prov1 = df_question_data.dropna(subset=['list'])#.fillna(0)
  #Selecionando onde Test não é NaN
  #df_prov2 = df_question_data.dropna(subset=['test'])#.fillna(0)
  #Concatenação para permitir um loc que busque em todas as instancias
  pat1 = '|'.join(['({})'.format(re.escape(c)) for c in list(df_lop_lists['id_list'])])
  #pat2 = '|'.join(['({})'.format(re.escape(c)) for c in provas_escolhidas])
  #Busca
  df_prov1 = df_prov1.loc[df_prov1['id_list'].str.contains(pat1)].copy()
  #df_prov2 = df_prov2.loc[df_prov2['test'].str.contains(pat2)].copy()
  #return pd.concat([df_prov1, df_prov2],axis=0).copy()
  return df_prov1

In [ ]:
def df_difficulty_performance (turma, prova = False):

  df_submissions = lop_submission(pd.DataFrame(turma, columns = ['id_class'])) # carregando dados de submissões da turma
  df_questions = select_questions(question_data(),listas_escolhidas,provas_escolhidas) # carregando dados de questões selecionadas
  
  if (prova): # filtro de provas 
    df_questions = df_questions.dropna(subset=['test']).fillna(0)
  else:
    df_questions = df_questions.dropna(subset=['list']).fillna(0)

  df_geral = pd.merge(df_submissions,df_questions, on = "question", how = "inner") # juntando os dataframes de submissões e questões
  df_difficulty_performance = df_geral.groupby(['user','registration','question','difficulty'])['hitPercentage'].max().reset_index() # pegando o máximo de HitPercentage de cada aluno
  df_difficulty_performance = df_difficulty_performance.groupby(['user','registration','difficulty'])['hitPercentage'].sum().reset_index() # fazendo a soma dos máximos
  df_difficulty_performance.rename(columns={'hitPercentage':'totalHitPercentage'},inplace = True) # renomeando coluna 'hitPercentage'

  df_questions = df_questions.groupby(['question','difficulty']).count().reset_index() # agrupando por questão e dificuldade
  df_questions = df_questions.groupby(['difficulty'])['question'].count().reset_index() # contando quantas questões por dificuldade existem
  df_questions.rename(columns={'question':'totalQuestions'},inplace=True) # renomeando coluna 'question'


  df_difficulty_performance = addData(df_difficulty_performance, df_difficulty_performance['difficulty'].unique(), df_difficulty_performance['user'].unique()  ) # chamando função para adicionar dados faltantes
  df_difficulty_performance = df_difficulty_performance.merge(df_questions, on = 'difficulty', how = 'outer') # juntando o dataframe principal com o de contagem de questões
  df_difficulty_performance.replace(np.NaN, 0 , inplace = True)
  df_difficulty_performance['averageDifficulty'] = df_difficulty_performance['totalHitPercentage'] / df_difficulty_performance['totalQuestions'] # adicionando coluna da média por dificuldade
  df_difficulty_performance = df_difficulty_performance.sort_values(['user','difficulty']) # ordernando o dataframe por nome e dificuldade

  return df_difficulty_performance

In [ ]:
# como a função é chamada : performance_difficulty_list_test(df_submission, df_questions_selected,'list')

def performance_difficulty_list_test(df_submission, df_questions_selected, listORtest = 'list'):
  #Selecionando apenas as questões de lista
  if listORtest == 'list':
    df_questions_selected = df_questions_selected.dropna(subset=['list'])
  #Selecionando apenas as questões de prova
  elif listORtest == 'test':
    df_questions_selected = df_questions_selected.dropna(subset=['test'])
  #Porcentagem máxima por questão
  df_performance = df_submission.groupby(['user','registration','question'])['hitPercentage'].max().reset_index()
  #Questão e a sua dificuldade associada
  df_questions_selected = df_questions_selected.groupby(['difficulty','question'])['id_list'].count().reset_index().copy()
  #Contando numero de questoes
  df_total_questions = df_questions_selected.groupby(['difficulty'])['question'].count().reset_index(name='totalQuestions')
  #Merge
  df_performance = pd.merge(df_performance, df_questions_selected.drop(columns='id_list'), on='question')
  #Somando as porcentagens de acerto por dificuldade para cada aluno
  df_performance = df_performance.groupby(['user','registration','difficulty'])['hitPercentage'].sum().reset_index(name='totalHitPercentage')
  #Conversão de float para inteiro
  df_performance['difficulty'] = df_performance['difficulty'].astype('int')
  #Lista de alunos
  df_without_duplicates = df_submission.drop_duplicates(subset=['registration'])
  df_without_duplicates = df_without_duplicates[['user','registration']]
  #Laço para fazer que todo aluno tenha as 5 dificuldades
  df_prov = pd.DataFrame()
  for difficulty in range(1, 6):
    df_users = df_without_duplicates.copy()
    df_users['difficulty'] = difficulty
    df_prov = df_prov.append(df_users)
  #Unindo e fazendo que apareça todas as dificuldades
  df_performance = pd.merge(df_performance,df_prov, on=['user','registration','difficulty'], how='outer').sort_values(by=['user','registration'])#.fillna(0)
  df_performance = pd.merge(df_performance, df_total_questions, on='difficulty', how='outer')#.fillna(0)
  #Calculando a média por dificuldade
  df_performance['mediaDifficulty'] = df_performance['totalHitPercentage'] / df_performance['totalQuestions']
  return df_performance.sort_values(by=['user','registration'])#.fillna(0)

In [ ]:
performance_difficulty_list_test(df_submission, df_questions, 'list' )

In [ ]:
# como a função é chamada: graph_more_less_difficulty_list_test_class(df_performance_difficulty_list, df_class, data, value = 70.0)

def graph_more_less_difficulty_list_test_class(df_performance_difficulty_list_test, df_class, data, value = 70.0):#value == 70 ou outro numero, data == turma em df
  #Aqui eu conto, por dificuldade, quantos tiraram mais de % de acerto escolhida
  df_more = df_performance_difficulty_list_test[df_performance_difficulty_list_test['mediaDifficulty'] >= value].groupby('difficulty')['mediaDifficulty'].count().reset_index(name='more')
  #E nesse conta quantos tiveram menos da % escolhida
  df_less = df_performance_difficulty_list_test[df_performance_difficulty_list_test['mediaDifficulty'] < value].groupby('difficulty')['mediaDifficulty'].count().reset_index(name='less')
  #Aqui junta ambos os dataframes, usando o outer, que, se por acaso não tiver uma ocorrência de uma lista em um dos dataframes, mesmo assim eles vão ser adicionados, é união dos conjuntos
  df_less_more = pd.merge(df_more, df_less, on = 'difficulty', how = 'outer')
  #E, se caso acontecer a situação de um não existir, substituimos o nan por 0
  df_less_more.replace(np.NaN, 0, inplace = True)
    #Aqui junta ambos os dataframes, usando o outer, que, se por acaso não tiver uma ocorrência de uma lista em um dos dataframes, mesmo assim eles vão ser adicionados, é união dos conjuntos
  df_less_more = pd.merge(df_more, df_less, on =  'difficulty', how = 'outer')
  #E, se caso acontecer a situação de um não existir, substituimos o nan por 0
  df_less_more.replace(np.NaN, 0, inplace = True)
  #Todas as dificuldades, para caso uma turma não tenha cadastrado todas, aparecer na grafico
  difficultys = pd.DataFrame([1,2,3,4,5], columns = ['difficulty'])
  df_less_more = pd.merge(df_less_more, difficultys, on = 'difficulty', how = 'outer').fillna(0)
  #Total de alunos na turma
  total = df_class.loc[df_class['id_class'] == data.iloc[0,0],'studentsCount'].max()
  #Faltantes
  df_less_more['missing'] = total - (df_less_more['more'] + df_less_more['less'])
  #Conversão para inteiro
  df_less_more = df_less_more.astype(int)
  #Renomeando 
  df_less_more = df_less_more.replace({'difficulty': {1:"Muito Fácil",2:"Fácil", 3:"Médio",4:"Difícil",5:"Muito Difícil"} })
  return df_less_more#.to_json(force_ascii=False, orient='records')

In [ ]:
# como a função é chamada : graph_performance_student_difficulty_list_test(df_performance_difficulty_list, df_questions_selected)

def graph_performance_student_difficulty_list_test(df_performance_difficulty_list_test,df_questions_selected):
  #Total de assuntos
  total_difficulties = df_questions_selected['difficulty'].value_counts().count()
  #Media geral por aluno
  df_student_media = (df_performance_difficulty_list_test.groupby(['registration'])['mediaDifficulty'].sum() / total_difficulties).reset_index(name='studentMediaDifficulty')
  #Retirando colunas desnecessárias
  df_graph = df_performance_difficulty_list_test.sort_values(by='user').drop(columns=['totalHitPercentage','totalQuestions'])
  #Merge
  df_graph = pd.merge(df_graph, df_student_media, on='registration')
  #Ordenando
  df_graph.sort_values(by=['user','registration','difficulty'], inplace = True)
  return df_graph.fillna(0)#.to_json(force_ascii=False,orient='records')

In [ ]:
# como a função é chamada : media_graph_performance_student_difficulty_list_test(df_performance_difficulty_list,data, df_class)

def media_graph_performance_student_difficulty_list_test(df_performance_difficulty_list_test,data, df_class):
  #Número de alunos na turma
  total_students = df_class.loc[df_class['id_class'].str.contains(data.iloc[0,0]),'studentsCount'].max()
  #Media (dos que fizeram ao menos 1 submissão) por dificuldade na turma
  df_mean_difficulty = (df_performance_difficulty_list.groupby(['difficulty'])['mediaDifficulty'].sum() / total_students).reset_index(name='mediaDifficultyClass')
  return df_mean_difficulty

In [ ]:
def graph_days_spent_difficulty(df_submission,df_question_data,df_lop_lists):#Histogram
  #Fazendo o merge para pegar a coluna dificuldade
  df_submission = df_submission.merge(select_questions(df_question_data, df_lop_lists)[:][['question','difficulty']], on = 'question', how = 'inner')
  #Convertendo para datetime
  df_submission['createdAt'] = pd.to_datetime(df_submission['createdAt'])
  #Criando campo com a data
  df_submission['dateSubmission'] = df_submission['createdAt'].dt.date
  #Conversão de ms para s
  df_submission['timeInSecounds'] = df_submission['timeConsuming'].divide(1000).astype(int)
  #Dataframe sem duplicatas e com tempo maior que 0
  df_without_duplicates = df_submission[df_submission["timeInSecounds"] > 0].drop(['environment','hitPercentage','timeConsuming','createdAt','timeInSecounds','char_change_number'], axis=1).drop_duplicates()
  #Calculando em quantos dias diferentes a dificuldade foi submetida por aluno
  df_different_days_difficulties = df_without_duplicates.groupby(['user','registration','difficulty'])['dateSubmission'].count().reset_index(name='differentDaysList')
  return df_different_days_difficulties#.to_json("graph_days_spent_difficulty.json",force_ascii=False,orient='records')


In [ ]:
# 1: Assume-se que a coluna numérica de interesse p/ calcular a média é sempre a última, e não pode haver mais de uma coluna numérica de média no mesmo df.
# 2: Assume-se que a variável "ordered_dfs_list" terá todos os seus dataframes se tratando da mesma coisa, e com as mesmas colunas.
# 3: Deve-se chamar a função da seguinte forma: graphWeightedAverage([df1.copy(),df2.copy()]). Caso não seja atribuido uma lista de pesos (ordered_weight_list = [x,y,z], se faz uma média aritmética.)

def graphWeightedAverage(ordered_dfs_list,ordered_weight_list=None,subjects=subjects.copy()):

  number_of_dfs = len(ordered_dfs_list) # calcula o número de dfs passados em ordered_dfs_list

  initial_column_names = ordered_dfs_list[0].columns.tolist() # armazena o nome das colunas no df antes de manipular
  last_column_name = initial_column_names[-1] # armazena o nome da última coluna (que é a numérica de interesse p/ calcular a média)

  # Etapa para checar se tá tudo ok com os pesos, e atribuir um valor default caso não tenha sido especificado
  if ordered_weight_list == None:
    ordered_weight_list = [0.5]*number_of_dfs # default
  elif number_of_dfs != len(ordered_weight_list):
      raise ValueError('Both lists must have the same length')
      return

  for i in range(0,number_of_dfs): # varrendo todos os dfs da lista (um for bem pequeno. Atualmente, apenas 2 loops)
    if 'registration' in ordered_dfs_list[i].columns.tolist(): # etapa necessária para lidar com o problema do lop de conseguir inserir caractere na matricula
      ordered_dfs_list[i]["registration"] = ordered_dfs_list[i]["registration"].astype(str)

    #ordered_dfs_list[i]["df_id"] = i
    ordered_dfs_list[i]["weight"] = ordered_weight_list[i] # atribui o respectivo peso para cada linha do df

    if i < 1: # concatena todos os dfs da lista
      df = ordered_dfs_list[0]
    else:
      df = pd.concat([df,ordered_dfs_list[i]])


  df[last_column_name] = df[last_column_name] * df["weight"] # multiplica a coluna de interesse pelo peso

  #as etapas a seguir são necessárias para eliminar o problema de haver assuntos na lista de exercícios e não haver na lista de provas, por exemplo.
  #desse modo, caso exista em um e não no outro, ignora-se o peso do elemento faltante no cálculo da média.
  student_weight_list = df.groupby(initial_column_names[:-1])["weight"].sum().reset_index().iloc[:,-1] 
  df = df.groupby(initial_column_names[:-1])[last_column_name].sum().reset_index()
  df[last_column_name] = df[last_column_name]/student_weight_list

  return df#.to_json("df_weighted_average_all_subjects.json",force_ascii=False, orient='records')

In [105]:
data = pd.DataFrame(['8394d1cb-86b5-4b90-bc62-4b9b192a81f6'], columns = ['id_class'])

In [106]:
df_question_data = question_data()

In [107]:
df_lop_lists = lop_lists(turma)

In [108]:
df_questions_selected = select_questions(df_question_data, df_lop_lists)

In [110]:
def performance_difficulty_list_test(df_submission, df_questions_selected, listORtest = 'list'):
  #Selecionando apenas as questões de lista
  if listORtest == 'list':
    df_questions_selected = df_questions_selected.dropna(subset=['list'])
  #Selecionando apenas as questões de prova
  elif listORtest == 'test':
    df_questions_selected = df_questions_selected.dropna(subset=['test'])
  #Porcentagem máxima por questão
  df_performance = df_submission.groupby(['user','registration','question'])['hitPercentage'].max().reset_index()
  #Questão e a sua dificuldade associada
  df_questions_selected = df_questions_selected.groupby(['difficulty','question'])['id_list'].count().reset_index().copy()
  #Contando numero de questoes
  df_total_questions = df_questions_selected.groupby(['difficulty'])['question'].count().reset_index(name='totalQuestions')
  #Merge
  df_performance = pd.merge(df_performance, df_questions_selected.drop(columns='id_list'),on='question')
  #Somando as porcentagens de acerto por dificuldade para cada aluno
  df_performance = df_performance.groupby(['user','registration','difficulty'])['hitPercentage'].sum().reset_index(name='totalHitPercentage')
  #Conversão de float para inteiro
  df_performance['difficulty'] = df_performance['difficulty'].astype('int')
  #Lista de alunos
  df_without_duplicates = df_submission.drop_duplicates(subset=['registration'])
  df_without_duplicates = df_without_duplicates[['user','registration']]
  #Laço para fazer que todo aluno tenha as 5 dificuldades
  df_prov = pd.DataFrame()
  for difficulty in range(1, 6):
    df_users = df_without_duplicates.copy()
    df_users['difficulty'] = difficulty
    df_prov = df_prov.append(df_users)
  #Unindo e fazendo que apareça todas as dificuldades
  df_performance = pd.merge(df_performance,df_prov, on=['user','registration','difficulty'], how='outer').sort_values(by=['user','registration'])#.fillna(0)
  df_performance = pd.merge(df_performance, df_total_questions, on='difficulty', how='outer')#.fillna(0)
  #Calculando a média por dificuldade
  df_performance['mediaDifficulty'] = df_performance['totalHitPercentage'] / df_performance['totalQuestions']
  return df_performance.sort_values(by=['user','registration'])#.fillna(0)

In [111]:
df_submission = lop_submission(pd.DataFrame(turma, columns = ['id_class']))

In [112]:
df_performance = df_submission.groupby(['user','registration','question'])['hitPercentage'].max().reset_index()

,user,registration,question,hitPercentage
0,Alanderson Dias da Silva,20200079821,Altura abaixo da média,100.0
1,Alanderson Dias da Silva,20200079821,Altura acima de média,100.0
2,Alanderson Dias da Silva,20200079821,Calculando expressões - parte 1,100.0
3,Alanderson Dias da Silva,20200079821,Calculando expressões - parte 2,100.0
4,Alanderson Dias da Silva,20200079821,Calculando expressões - parte 3,100.0
...,...,...,...,...
4252,magnnwsb,20200082424,Troca de dígitos UCD,100.0
4253,magnnwsb,20200082424,Troca de dígitos UDC,100.0
4254,magnnwsb,20200082424,Troca de valores entre variáveis,100.0
4255,magnnwsb,20200082424,Área e Volume do círculo de raio R,100.0


In [113]:
df_questions_selected2 = df_questions_selected.groupby(['difficulty','question'])['id_list'].count().reset_index().copy()

,difficulty,question,id_list
0,1.0,Altura abaixo da média,1
1,1.0,Altura acima de média,1
2,1.0,Aprendendo a armazenar vários elementos em um ...,1
3,1.0,Aprendendo a usar repetição aninhada,1
4,1.0,Aprendendo a usar um Vetor,1
...,...,...,...
118,4.0,Qual o dia da semana?,1
119,4.0,Raízes de uma equação de segundo grau,1
120,4.0,Retirar repetidos,1
121,5.0,Data válida,1


In [129]:
df_total_questions = df_questions_selected2.groupby(['difficulty'])['question'].count().reset_index(name='totalQuestions')

,difficulty,totalQuestions
0,1.0,35
1,2.0,48
2,3.0,32
3,4.0,6
4,5.0,2


In [130]:
df_performance2 = pd.merge(df_performance, df_questions_selected2.drop(columns='id_list') , on='question')

,user,registration,question,hitPercentage,difficulty
0,Alanderson Dias da Silva,20200079821,Altura abaixo da média,100.0,1.0
1,Ana Augusta de Carvalho Rangel,20200076463,Altura abaixo da média,100.0,1.0
2,Angelo Ryan Cavalcante Nunes,20200068800,Altura abaixo da média,100.0,1.0
3,Antonio Amâncio,20190073238,Altura abaixo da média,100.0,1.0
4,Arinaldo Alves Filho,20200071529,Altura abaixo da média,100.0,1.0
...,...,...,...,...,...
4252,Jhosef Nascimento,20200096662,Número primo ou não?,100.0,3.0
4253,Jhosef Nascimento,20200096662,Pesando os bois,40.0,2.0
4254,Jhosef Nascimento,20200096662,Preenchimento de vetor I,100.0,1.0
4255,Jhosef Nascimento,20200096662,Soma de fatoriais,100.0,5.0


In [131]:
df_performance3 = df_performance2.groupby(['user','registration','difficulty'])['hitPercentage'].sum().reset_index(name='totalHitPercentage')

,user,registration,difficulty,totalHitPercentage
0,Alanderson Dias da Silva,20200079821,1.0,2200.00
1,Alanderson Dias da Silva,20200079821,2.0,2100.00
2,Alanderson Dias da Silva,20200079821,3.0,1500.00
3,Alanderson Dias da Silva,20200079821,4.0,400.00
4,Alanderson Dias da Silva,20200079821,5.0,87.50
...,...,...,...,...
364,eduardo guedes de moura calixto,20200001435,5.0,100.00
365,magnnwsb,20200082424,1.0,1433.33
366,magnnwsb,20200082424,2.0,1300.00
367,magnnwsb,20200082424,3.0,300.00


In [132]:
  df_performance3['difficulty'] = df_performance3['difficulty'].astype('int')


,user,registration,difficulty,totalHitPercentage
0,Alanderson Dias da Silva,20200079821,1,2200.00
1,Alanderson Dias da Silva,20200079821,2,2100.00
2,Alanderson Dias da Silva,20200079821,3,1500.00
3,Alanderson Dias da Silva,20200079821,4,400.00
4,Alanderson Dias da Silva,20200079821,5,87.50
...,...,...,...,...
364,eduardo guedes de moura calixto,20200001435,5,100.00
365,magnnwsb,20200082424,1,1433.33
366,magnnwsb,20200082424,2,1300.00
367,magnnwsb,20200082424,3,300.00


In [133]:
df_without_duplicates = df_submission.drop_duplicates(subset=['registration'])
df_without_duplicates = df_without_duplicates[['user','registration']]

,user,registration
0,Josué Anuar,20200073425
2,Sara Moreira,20200078904
3,Nathan Telles,20180146330
5,Jean Magnus,20200080063
6,Natália,20200073882
...,...,...
10440,Toane Lima,20200082200
11113,Igor Rosberg Aluno,00000000
13446,Fabio Gomes da Silva,20200075887
13660,Jardean Fernandes,20200076104


In [134]:
df_prov = pd.DataFrame()


In [135]:
for difficulty in range(1, 6):
    df_users = df_without_duplicates.copy()
    df_users['difficulty'] = difficulty
    df_prov = df_prov.append(df_users)

,user,registration,difficulty
0,Josué Anuar,20200073425,1
2,Sara Moreira,20200078904,1
3,Nathan Telles,20180146330,1
5,Jean Magnus,20200080063,1
6,Natália,20200073882,1
...,...,...,...
10440,Toane Lima,20200082200,5
11113,Igor Rosberg Aluno,00000000,5
13446,Fabio Gomes da Silva,20200075887,5
13660,Jardean Fernandes,20200076104,5


In [136]:
df_performance4 = pd.merge(df_performance3,df_prov, on=['user','registration','difficulty'], how='outer').sort_values(by=['user','registration'])#.fillna(0)

,user,registration,difficulty,totalHitPercentage
0,Alanderson Dias da Silva,20200079821,1,2200.00
1,Alanderson Dias da Silva,20200079821,2,2100.00
2,Alanderson Dias da Silva,20200079821,3,1500.00
3,Alanderson Dias da Silva,20200079821,4,400.00
4,Alanderson Dias da Silva,20200079821,5,87.50
...,...,...,...,...
365,magnnwsb,20200082424,1,1433.33
366,magnnwsb,20200082424,2,1300.00
367,magnnwsb,20200082424,3,300.00
368,magnnwsb,20200082424,4,100.00


In [137]:
df_performance5 = pd.merge(df_performance4, df_total_questions, on='difficulty', how='outer')#.fillna(0)

,user,registration,difficulty,totalHitPercentage,totalQuestions
0,Alanderson Dias da Silva,20200079821,1,2200.0,35
1,Ana Augusta de Carvalho Rangel,20200076463,1,1700.0,35
2,Angelo Ryan Cavalcante Nunes,20200068800,1,2200.0,35
3,Anthony da costa aquino,20180029737,1,1000.0,35
4,Antonio Amâncio,20190073238,1,2200.0,35
...,...,...,...,...,...
430,Wellington Toldo,20200070147,5,NaN,2
431,bernardo leitao da costa pinto,20200080662,5,100.0,2
432,bruno,20200102705,5,NaN,2
433,eduardo guedes de moura calixto,20200001435,5,100.0,2


In [156]:
df_performance5['mediaDifficulty'] = df_performance5['totalHitPercentage'] / df_performance5['totalQuestions']

,user,registration,difficulty,totalHitPercentage,totalQuestions,mediaDifficulty
378,Jhosef Nascimento,20200096662,5,200.0,2,100.000000
30,Jhosef Nascimento,20200096662,1,3200.0,35,91.428571
204,Jhosef Nascimento,20200096662,3,2900.0,32,90.625000
299,Larissa de Alcântara Pereira,20200072885,4,500.0,6,83.333333
346,eduardo guedes de moura calixto,20200001435,4,500.0,6,83.333333
...,...,...,...,...,...,...
428,Viviane Brasil,20200070746,5,NaN,2,NaN
429,WALLYSON MATHEUS SOUZA DE OLIVEIRA,20200083341,5,NaN,2,NaN
430,Wellington Toldo,20200070147,5,NaN,2,NaN
432,bruno,20200102705,5,NaN,2,NaN


In [ ]:
# como a função é chamada: graph_more_less_difficulty_list_test_class(df_performance_difficulty_list, df_class, data, value = 70.0)

def graph_more_less_difficulty_list_test_class(df_performance_difficulty_list_test, df_class, data, value = 70.0):#value == 70 ou outro numero, data == turma em df
  #Aqui eu conto, por dificuldade, quantos tiraram mais de % de acerto escolhida
  df_more = df_performance_difficulty_list_test[df_performance_difficulty_list_test['mediaDifficulty'] >= value].groupby('difficulty')['mediaDifficulty'].count().reset_index(name='more')
  #E nesse conta quantos tiveram menos da % escolhida
  df_less = df_performance_difficulty_list_test[df_performance_difficulty_list_test['mediaDifficulty'] < value].groupby('difficulty')['mediaDifficulty'].count().reset_index(name='less')
  #Aqui junta ambos os dataframes, usando o outer, que, se por acaso não tiver uma ocorrência de uma lista em um dos dataframes, mesmo assim eles vão ser adicionados, é união dos conjuntos
  df_less_more = pd.merge(df_more, df_less, on = 'difficulty', how = 'outer')
  #E, se caso acontecer a situação de um não existir, substituimos o nan por 0
  df_less_more.replace(np.NaN, 0, inplace = True)
    #Aqui junta ambos os dataframes, usando o outer, que, se por acaso não tiver uma ocorrência de uma lista em um dos dataframes, mesmo assim eles vão ser adicionados, é união dos conjuntos
  df_less_more = pd.merge(df_more, df_less, on =  'difficulty', how = 'outer')
  #E, se caso acontecer a situação de um não existir, substituimos o nan por 0
  df_less_more.replace(np.NaN, 0, inplace = True)
  #Todas as dificuldades, para caso uma turma não tenha cadastrado todas, aparecer na grafico
  difficultys = pd.DataFrame([1,2,3,4,5], columns = ['difficulty'])
  df_less_more = pd.merge(df_less_more, difficultys, on = 'difficulty', how = 'outer').fillna(0)
  #Total de alunos na turma
  total = df_class.loc[df_class['id_class'] == data.iloc[0,0],'studentsCount'].max()
  #Faltantes
  df_less_more['missing'] = total - (df_less_more['more'] + df_less_more['less'])
  #Conversão para inteiro
  df_less_more = df_less_more.astype(int)
  #Renomeando 
  df_less_more = df_less_more.replace({'difficulty': {1:"Muito Fácil",2:"Fácil", 3:"Médio",4:"Difícil",5:"Muito Difícil"} })
  return df_less_more#.to_json(force_ascii=False, orient='records')

In [162]:
graph_more_less_difficulty_list_test_class(df_performance5,df_class, data, value = 50.0)

,difficulty,more,less,missing
0,Muito Fácil,44,43,4
1,Fácil,7,79,5
2,Médio,13,71,7
3,Difícil,48,35,8
4,Muito Difícil,23,6,62
